In [ ]:
import os
import zipfile
import requests
import numpy as np
import tensorflow as tf

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.rcParams["figure.figsize"] = (10, 10)

%matplotlib inline

In [ ]:
# IoU Calculation
def intersection_over_union(dbox_data, area=None):
    """
    Calculates IoU of the 0th indexed bounding box with the rest of the boxes ([1:]).

    Args:
        dbox_data (tensor): bounding boxes.
                    Format: [
                                [x_min, y_min, x_max, y_max, ...],
                                [x_min, y_min, x_max, y_max, ...],
                                         :
                                         :
                                [x_min, y_min, x_max, y_max, ...],
                            ]

        area: Area of the bounding boxes. Default: None.
              If area is None, the function calculates the area and use it.
              Foramt: [area_of_bounding_box_1, area_of_bounding_box_2, ....]

    Returns:
        (tensor): IoU of the 0th indexed bounding box with the rest of the boxes. Length will be one less than
                     the number of bounding box.
    """

    # Calculate area if not provided.
    if area is None:

      # Calculate width and height
      width = dbox_data[:, 2] - dbox_data[:, 0] +
      height = dbox_data[:, 3] - dbox_data[:, 1] + 1

      area = tf.multiply(width, height)

    # Identify overlap box ccordinates
    x_min = tf.math.maximum(dbox_data[0, 0], dbox_data[1:, 0]) # left boundary of intersection
    y_min = tf.math.maximum(dbox_data[0, 1], dbox_data[1:, 1]) # top boundary
    x_max = tf.math.minimum(dbox_data[0, 2], dbox_data[1:, 2]) # right boundary
    y_max = tf.math.minimum(dbox_data[0, 3], dbox_data[1:, 3]) # bottom boundary

    # Calculate width and height of overlapped regions and suppress negative values.
    intersection_width  = tf.math.maximum((x_max - x_min + 1), 0.0)
    intersection_height = tf.math.maximum((y_max - y_min + 1), 0.0)

    # IoU calculation.
    intersection_area = intersection_width * intersection_height
    iou = intersection_area / (area[0] + area[1:] - intersection_area)

    return iou

In [ ]:
def nms_rescoring(dbox_data, score_threshold=0.001, iou_threshold=0.3,  soft_nms=True, rescoring='Linear', sigma=0.5):
    """
        Soft-NMS TensorFlow implementation

        Parameters:
            dbox_data: coordinates of the detected boxes and their scores [x1, y1, x2, y2, confidence_score]

            sigma: parameter of the Gaussian function

            iou_threshold: Intersection over Union threshold (for the linear method)

            score_threshold: confidence score threshold

            soft_nms: If True, soft NMS will be used else har-NMS will be used.

            rescoring: an integer value 0 or 1, where 1 corresponds to Gaussian rescoring method and 0 to
                       the linear method

        Return value:
            array of bounding boxes with corresponding recalculated scores (in accordance with the applied method)
    """

    # Get bounding box coordinates.
    xmin_initial = dbox_data[:, 0]
    ymin_initial = dbox_data[:, 1]
    xmax_initial = dbox_data[:, 2]
    ymax_initial = dbox_data[:, 3]

    areas = (xmax_initial - xmin_initial + 1) * (ymax_initial - ymin_initial + 1)

    # Concatinate area with the tensor
    dbox_data = tf.concat([dbox_data, areas[:, None]], axis=1)

    # Converting dbox_data to type tf.Variable as the contents and shape of the matrix will change.
    dbox_data = tf.Variable(dbox_data, shape=tf.TensorShape(None))

    # List for the result boxes.
    final_dbox = []

    while tf.shape(dbox_data)[0] > 0:
        # Position of detection box with maximum confidence score.
        max_index = tf.math.argmax(dbox_data[:, 4], axis=0)

        # Swap bounding box present at index 0 with bounding box at max_index position.
        values = dbox_data.gather_nd([[max_index], [0]]) # BBox at max_index and index 0
        dbox_data.scatter_nd_update(indices=[[0], [max_index]], updates=values) # Swap

        # Add max score box to the final result.
        final_dbox.append(dbox_data[0, :-1])

        # Get area to use in Intersection over Union calculation.
        current_areas = dbox_data[:, -1]

        # Get IoUs between the highest confident bounding box and the rest of the bounding boxes.
        iou = intersection_over_union(dbox_data, area=current_areas)

        # Boolean tensor for which IoU scores > iou_threshold.
        iou_thresh_ids = iou > iou_threshold

        # Soft-NMS.
        if soft_nms:
            # Score recalculation with linear method
            if rescoring.lower() == 'linear':
                # Init score tensor as ones. Later it will be multiplied by the actual score (confidence).
                score = tf.ones_like(iou, dtype=tf.float32)

                # Filter IoU values greater than the threshold.
                iou_tensor = iou[iou_thresh_ids]

                # Get the scores, where IoU is more than iou_threshold.
                score_tensor = score[iou_thresh_ids]

                # Update formulation: s_i <-- s_i * (1 - iou_i)
                # However, here we are calculating (1 - iou_i). Later it will be multiplied by s_i.
                # score_tensor is a vector of 1's
                score_tensor -= iou_tensor

                # Update only those scores where iou  IoU > iou_threshold.
                # Here, score = 1 - IoU if IoU > iou_threshold else 1.
                updated_scores = tf.tensor_scatter_nd_update(score, tf.where(iou > iou_threshold), score_tensor)

            # Score recalculation with Gaussian method.
            elif rescoring.lower() == 'gaussian':
                updated_scores = tf.math.exp(-(iou * iou) / sigma)

            else:
                raise ValueError("rescoring needs to be either 'Linear' or 'Gaussian'")

        # NMS (hard-NMS)
        else:
            score = tf.ones_like(iou, dtype=tf.float32)

            # Filter score values for which IoU greater than the threshold.
            score_tensor = score[iou_thresh_ids]
            score_tensor = tf.zeros_like(score_tensor, dtype=tf.float32)

            # Update only those scores where iou is greater than the threshold.
            updated_scores = tf.tensor_scatter_nd_update(score, tf.where(iou_thresh_ids), score_tensor)

        # Multiply by s_i to complete the soft-NMS.
        # It also works for hard-NMS because multiply zero by anything is fine.
        current_scores = dbox_data[1:, 4]
        dbox_data[1:, 4].assign(current_scores * updated_scores)

        # After the score update, get the bounding box indices that score is greater than score_threshold.
        # Note that the starting index is 1, not 0. Because index zero is already a qualified bounding box.
        scores = dbox_data[1:, 4]
        final_box_pos = tf.where(scores >= score_threshold)

        # Note that final_box_pos gives indices of the bounding box candidate (that qualifies the score threshold).
        # However, we also need bounding boxes, the .gather_nd(...) get items at indices final_box_pos.
        # Note that one is added to final_box_pos because these indices came from dbox_data[1:], not dbox_data.
        dbox_data.assign(dbox_data.gather_nd(indices=(final_box_pos + 1)))

    # Convert to tf tensor before returning.
    return tf.stack(final_dbox)


In [ ]:
# Check implementation
def download_file(url, save_name):
  if not os.path.exists(save_name):
    file = requests.get(url)
    open(save_name, 'wb').write(file.content)

In [ ]:
def unzip(zip_file=None):
  try:
    with zipfile.ZipFile(zip_file) as z:
      z.extractall("./")
      print("Extracetd all")
  except:
    print("Invalid file")

In [ ]:
download_file(
    'https://www.dropbox.com/s/ryglj9p4pk4eu7s/data.zip?dl=1',
    'data.zip'
)

unzip(zip_file='data.zip')

In [ ]:
# Plotting thr bounding box

In [ ]:
def plot_bounding_boxes(img_path, dbox_data, clr=None, linewidth=2):
    """
    "Plot bounding boxes"

    Prameters:

    img_path (str): image path

    dbox_data (tensor): a tensor of bounding boxes.
                        Format: [
                             [x_min, y_min, x_max, y_max, ...],
                             [x_min, y_min, x_max, y_max, ...],
                                         :
                                         :
                             [x_min, y_min, x_max, y_max, ...],

                        ]
    clr (str): color code, for example, "r", "b", etc... Default is None.
               In this case color = ['r', 'g', 'b', 'c', 'm', 'y', 'k', 'w'] will be used one by one


    linewidth (int): The width of the bounding boxes lines that will be drawn on the image.


    """

    # Read image using PIL.Image.open .
    img = np.array(Image.open(img_path), dtype=np.uint8)


    # Calculate width and height for all boxes.
    # width = x_max - x_min + 1, x_max column index 2 and x_min column index is 0.
    # height = y_max - y_min + 1, y_max column index 3 and y_min column index is 1.

    width = dbox_data[:, 2] - dbox_data[:, 0] + 1
    height = dbox_data[:, 3] - dbox_data[:, 1] + 1

    color = ['r', 'b', 'g', 'c', 'm', 'y', 'k', 'w']
    fig = plt.figure(figsize=(8, 8))

    ax = fig.add_subplot(111, aspect='equal')

    # Display the image.
    ax.imshow(img)

    for i in range(tf.shape(dbox_data)[0]):
        ax.add_patch(
            patches.Rectangle(
                (dbox_data[i, 0], dbox_data[i, 1]),
                width[i],
                height[i],
                fill=False,
                color=color[i%len(color)] if clr is None else clr,
                linewidth = linewidth
            )
        )

    plt.show()

    return

In [ ]:
# The image path.
image_path = 'data/w8_c4-dog-bike-car.jpg'

# The bounding box data path.
bbox_path = 'data/w8_boxes.npy'

# Load bounding box file.
bboxes = np.load(bbox_path)

print('Sample of bounding boxes:\n')
print(bboxes[:5])


print('\nTotal no of bounding boxes: {}'.format(len(bboxes)))

In [ ]:
# Plot all bounding boxes.
plot_bounding_boxes(image_path, bboxes, clr='r', linewidth=1)

In [ ]:
def check_nms(img_path, bboxes, iou_thresh, score_thresh, clr = None):

    # Linear.
    print("Linear rescoring results:")
    bboxes_l_soft = nms_rescoring(dbox_data=bboxes, score_threshold=score_thresh,
                                  iou_threshold=iou_thresh, rescoring='Linear')
    print("No. of boxes",len(bboxes_l_soft))
    plot_bounding_boxes(img_path, bboxes_l_soft, clr = clr, linewidth=3)

    # Gaussian.
    print("Gaussian rescoring results:")
    bboxes_g_soft = nms_rescoring(dbox_data=bboxes, score_threshold=score_thresh,
                                  iou_threshold=iou_thresh, rescoring='Gaussian')
    print("No. of boxes",len(bboxes_g_soft))
    plot_bounding_boxes(img_path, bboxes_g_soft, clr = clr, linewidth=3)


    # Hard-NMS.
    print("Hard NMS results:")
    bboxes_h_nms = nms_rescoring(dbox_data=bboxes, score_threshold=score_thresh,
                                 iou_threshold=iou_thresh, soft_nms=False)
    print("No. of boxes",len(bboxes_h_nms))
    plot_bounding_boxes(img_path, bboxes_h_nms, clr = clr, linewidth=3)

    return

In [ ]:
check_nms(img_path=image_path, bboxes=bboxes, iou_thresh=0.24, score_thresh=0.2)

In [ ]:
image_path = 'data/image.jpg'

# The bounding box data path.
bbox_path = 'data/bboxes_new.npy'

# Load the bounding box file.
bboxes = np.load(bbox_path).astype(np.float32)


print('Total no of bounding boxes: {}'.format(len(bboxes)))

In [ ]:
plot_bounding_boxes(image_path, bboxes, clr='r', linewidth=2)

In [ ]:
check_nms(image_path, bboxes, iou_thresh=0.5, score_thresh=0.15)

In [ ]:
image_path = 'data/image1.jpg'

# The bounding box data path.
bbox_path = 'data/boxes_1.npy'

# Load the bounding box file.
bboxes = np.load(bbox_path).astype(np.float32)

In [ ]:
plot_bounding_boxes(image_path, bboxes, clr='r', linewidth=2)

In [ ]:
image_path = 'data/image2.jpg'

# The bounding box data path.
bbox_path = 'data/boxes_2.npy'

# Load the bounding box file.
bboxes = np.load(bbox_path).astype(np.float32)


print('Total no of bounding boxes: {}'.format(len(bboxes)))

In [ ]:
plot_bounding_boxes(image_path, bboxes, clr='r', linewidth=2)

In [ ]:
check_nms(image_path, bboxes, iou_thresh=0.4, score_thresh=0.27)

In [ ]:
image_path = 'data/image4.jpg'

# The bounding box data path.
bbox_path = 'data/boxes_4.npy'

# Load the bounding box file.
bboxes = np.load(bbox_path).astype(np.float32)

print('Total no of bounding boxes: {}'.format(len(bboxes)))

In [ ]:
plot_bounding_boxes(image_path, bboxes, clr='r', linewidth=2)

In [ ]:
check_nms(image_path, bboxes, iou_thresh=0.5, score_thresh=0.2, clr = 'b')